In [22]:
%env MLFLOW_TRACKING_URI=sqlite:///mlruns.db

env: MLFLOW_TRACKING_URI=sqlite:///mlruns.db


# 1. Adding dropout and normalization layers
Study the pytorch documentation for:
- Dropout https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
- normalization layers https://pytorch.org/docs/stable/nn.html#normalization-layers

Experiment with adding dropout and normalization layers to your model. Some rough guidelines where to add them relative to Linear or Conv2d layers:
- Dropout: after Linear or Conv2d layers. Often added after the last Linear layer *before* the output layer, but could occur more often.
- Normalization layers: right after (blocks of) Linear or Conv2d layers, but before activation functions.

In [23]:
from pathlib import Path
import torch
import torch.nn as nn
from loguru import logger
import warnings
warnings.simplefilter("ignore", UserWarning)
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import BasePreprocessor

fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

2024-09-21 17:34:07.970 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at C:\Users\leons\.cache\mads_datasets\fashionmnist
2024-09-21 17:34:07.971 | INFO     | mads_datasets.base:download_data:124 - File already exists at C:\Users\leons\.cache\mads_datasets\fashionmnist\fashionmnist.pt


using cuda


c:\Users\leons\OneDrive\Bureaublad\School\ML Trainer\.venv\lib\site-packages\mads_datasets\factories\torchfactories.py:71: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data

In [12]:
import torch
from torch import nn
from loguru import logger
from torchsummary import summary

# Define the CNN model
class CNN(nn.Module):
    def __init__(self, filters: int, units1: int, units2: int, input_size: tuple):
        super().__init__()
        self.in_channels = input_size[1]
        self.input_size = input_size

        self.convolutions = nn.Sequential(
            nn.Conv2d(self.in_channels, filters, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(kernel_size=2),  # Output size halved
            nn.Conv2d(filters, filters*2, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(filters*2),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(kernel_size=2),  # Output size halved again
            nn.Conv2d(filters*2, filters*3, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(filters*3),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.MaxPool2d(kernel_size=2),  # Output size halved once more
        )

        # Calculate the flattened size based on actual output shape after convolutions
        flattened_size = self._get_flattened_size(input_size)
        logger.info(f"Flattened size for the first Linear layer: {flattened_size}")

        # Remove AdaptiveAvgPool2d, as the tensor is already reduced
        self.dense = nn.Sequential(
            nn.Flatten(),  # Flatten the 2D to 1D
            nn.Linear(flattened_size, units1),  # Input size should match the flattened size
            nn.BatchNorm1d(units1),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(units1, units2),
            nn.BatchNorm1d(units2),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(units2, 10)  # Output 10 classes
        )

    # This function calculates the flattened size after convolutions
    def _get_flattened_size(self, input_size):
        x = torch.ones(1, *input_size[1:], dtype=torch.float32)  # Add batch dimension
        x = self.convolutions(x)
        logger.info(f"Output shape after convolutions: {x.shape}")
        return x.numel()  # Return the total number of elements (flattened size)

    def forward(self, x):
        x = self.convolutions(x)
        x = self.dense(x)  # Forward to dense layers
        return x

# Define the model
model = CNN(filters=128, units1=128, units2=64, input_size=(32, 1, 28, 28))

# Print the model summary
summary(model, input_size=(1, 28, 28), device='cpu')  # Correct input size for summary


2024-09-21 17:47:26.855 | INFO     | __main__:_get_flattened_size:53 - Output shape after convolutions: torch.Size([1, 384, 2, 2])
2024-09-21 17:47:26.864 | INFO     | __main__:__init__:33 - Flattened size for the first Linear layer: 1536


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 28, 28]           1,280
       BatchNorm2d-2          [-1, 128, 28, 28]             256
              ReLU-3          [-1, 128, 28, 28]               0
           Dropout-4          [-1, 128, 28, 28]               0
         MaxPool2d-5          [-1, 128, 14, 14]               0
            Conv2d-6          [-1, 256, 12, 12]         295,168
       BatchNorm2d-7          [-1, 256, 12, 12]             512
              ReLU-8          [-1, 256, 12, 12]               0
           Dropout-9          [-1, 256, 12, 12]               0
        MaxPool2d-10            [-1, 256, 6, 6]               0
           Conv2d-11            [-1, 384, 4, 4]         885,120
      BatchNorm2d-12            [-1, 384, 4, 4]             768
             ReLU-13            [-1, 384, 4, 4]               0
          Dropout-14            [-1, 38

: 

In [8]:
from mltrainer.trainer import TrainerSettings, ReportTypes
from mltrainer import metrics
log_dir = Path("../../models/cnn").resolve()
if not log_dir.exists():
    log_dir.mkdir(parents=True)
accuracy = metrics.Accuracy()
settings = TrainerSettings(
    epochs=10,
    metrics=[accuracy],
    logdir=log_dir,
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD],
)

In [9]:
import torch.optim as optim
from mltrainer import metrics
from mltrainer.trainer import Trainer
optimizer = optim.Adam
loss_fn = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()
trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optimizer,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau,
    device=device,
    )

2024-09-21 16:14:59.933 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to C:\Users\leons\OneDrive\Bureaublad\School\models\cnn\20240921-161459
2024-09-21 16:14:59.937 | INFO     | mltrainer.trainer:__init__:72 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.


In [10]:
trainer.loop()

100%|██████████| 937/937 [00:10<00:00, 93.60it/s]
2024-09-21 16:15:10.608 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 0.5297 test 0.5758 metric ['0.7952']
100%|██████████| 937/937 [00:09<00:00, 95.37it/s]
2024-09-21 16:15:21.089 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 0.3300 test 0.4091 metric ['0.8631']
100%|██████████| 937/937 [00:09<00:00, 95.38it/s]
2024-09-21 16:15:31.566 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 0.2884 test 0.3447 metric ['0.8783']
100%|██████████| 937/937 [00:09<00:00, 94.23it/s]
2024-09-21 16:15:42.164 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 0.2588 test 0.4650 metric ['0.8264']
2024-09-21 16:15:42.165 | INFO     | mltrainer.trainer:__call__:234 - best loss: 0.3447, current loss 0.4650.Counter 1/10.
100%|██████████| 937/937 [00:09<00:00, 94.46it/s]
2024-09-21 16:15:52.746 | INFO     | mltrainer.trainer:report:191 - Epoch 4 train 0.2424 test 0.4476 metric ['0.8360']
2024-09-21 16:15:52.747 | INFO  

----------
Personal notes!

- Dropout is tpyically used after the activation function and before the pooling function. This is because after the pooling function there are less activation neurons to drop out. so less weights to be influenced by the dropout. Dropout is designed to randomly set a fraction of the activations to zero. Applying it after the activation function ensures that the dropout is applied to the non-linear transformed features, which helps in regularizing the network more effectively.
Also to maintain sctructure of the network.

- Stabilizing Activations: By normalizing the activations before applying the non-linearity, you ensure that the inputs to the activation functions have a consistent distribution, which helps in stabilizing the training process.
Improving Gradient Flow: Normalization helps in maintaining a stable gradient flow through the network, which can prevent issues like vanishing or exploding gradients.
Accelerating Training: Normalized activations can lead to faster convergence during training, as the network can learn more efficiently.
    Why Not After Activation?
    Placing normalization layers after the activation functions can still work, but it might not be as effective. The primary goal of normalization is to control the distribution of the inputs to the activation functions, ensuring they are within a range that the activation functions can handle well.

- Adding only batch normalization layer to the model, the accuracy improved to 92.3%! re adding the dropout layers decreased performance slightly but should improve generalization.


# 2. Adding convolutional and pooling layers
Previous lessons, you have started to experiment with you model.
You might have tested the impact of the amount of units, the depth of layers and different learning rates.

This lesson, we have added some new types of layers: convolutional and pooling layers.
Experiment with adding these new layers.

Also, have a look at the `ModuleList`: https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html#modulelist
It can be really useful to create a list of layers from a configfile, and then use that list to create your model.
Instead of just adding a single layer, you could also add a block of layers (eg a Conv2d layer, followed by a ReLU layer, followed by a BatchNorm2d layer, followed by a MaxPool2d layer) and repeat that in a loop, adding it to the `ModuleList`.

# 3. Improve your pipeline
In addition to new layers, we have expanded our logging tools with MLFlow, so we currently can choose between gin-config, tensorboard and MLFlow.

Expand your training pipeline you started in the previous lesson such that:

- you can switch between models by changing a config file
- you can test different hyperparameters by changing a config file
- you automatically log settings: model picked, hyperparameters, metrics, etc. : use either gin-config, tensorboard or MLFlow to log that, or a combination, whatever you prefer.
- Important: doing a master means you don't just start engineering a pipeline, but you need to reflect. Why do you see the results you see? What does this mean, considering the theory? Write down lessons learned and reflections, based on experimental results.
- continuously improve your code: 
    - clean up your experimental environment, such that it doesnt get too messy
    - automate the boring stuff: use a Makefile, use configfiles, automate logging, etc.
    - use git: commit your changes often and with descriptive messages
    - separate code for pipelines, configs, models, modeltraining and results.